<a href="https://colab.research.google.com/github/utkarshrajputt/AI-ChatBot/blob/Utkarsh/CHATBOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qq langchain wget llama-index cohere llama-cpp-python

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 7.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.7/248.7 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.8/176.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.5 MB/s eta 0:00

In [3]:
import wget

def bar_custom(current, total, width=80):
    print("Downloading: %d%% [%d / %d] bytes" % (current / total * 100, current, total))

model_url="https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/blob/main/llama-2-7b-chat.Q2_K.gguf"
wget.download(model_url, bar=bar_custom)

Downloading: 0% [0 / 65876] bytes
Downloading: 12% [8192 / 65876] bytes
Downloading: 24% [16384 / 65876] bytes
Downloading: 37% [24576 / 65876] bytes
Downloading: 49% [32768 / 65876] bytes
Downloading: 62% [40960 / 65876] bytes
Downloading: 74% [49152 / 65876] bytes
Downloading: 87% [57344 / 65876] bytes
Downloading: 99% [65536 / 65876] bytes
Downloading: 100% [65876 / 65876] bytes


'llama-2-7b-chat.Q2_K (1).gguf'

In [5]:
!pip -q install streamlit

In [6]:
%%writefile app.py
import streamlit as st
from llama_index import(
    SimpleDirectoryReader,
   VectorStoreIndex,
    ServiceContext,
)
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import (
    messages_to_prompt,
    completion_to_prompt
)
from langchain.schema import (
    HumanMessage,
    SystemMessage,
    AIMessage
)

def init_page() -> None:
  st.set_page_config(
      page_title="Personal ChatBot",
      page_icon="🦙",
      layout="wide",
  )
  st.header("🦙 Personal ChatBot")
  st.sidebar.title("Options")

def select_llm() -> LlamaCPP:
  return LlamaCPP(
      model_path="llama-2-7b-chat.Q2_K.gguf",
      temperature=0.1,
      max_new_tokens=500,
      context_window=3900,
      generate_kwargs={},
      model_kwargs={"n_gpu_layers": 1},
      message_to_prompt=messages_to_prompt,
      completion_to_prompt=completion_to_prompt,
      verbose=True,
  )

  def init_messages() -> None:
    clear_button = st.sidebar.button("Clear Conversation", key="clear")
    if clear_button or "messages" not in st.session_state:
      st.session_state.messages = [
        SystemMessage(
          content="you are a helpful AI assistant. Reply your answer in markdown format."
        )
    ]

def get_answer(llm, messages) -> str:
  response = llm.complete(messages)
  return response.text

def main() -> None:
  init_page()
  llm = select_llm()
  init_messages()

  if user_input := st.chat_input("Input your question!"):
    st.session_state.messages.append(HumanMessage(content=user_input))
    with st.spinner("Bot is typing ..."):
      answer = get_answer(llm, user_input)
      print(answer)
    st.session_state.messages.append(AIMessage(content=answer))


  messages = st.session_state.get("messages", [])
  for message in messages:
    if isinstance(message, AIMessage):
      with st.chat_message("assistant"):
        st.markdown(message.content)
    elif isinstance(message, HumanMessage):
      with st.chat_message("user"):
        st.markdown(message.content)

if __name__ == "__main__":
  main()




Writing app.py


In [7]:
!streamlit run app.py & npx localtunnel --port 8501



Need to install the following packages:
  localtunnel@2.0.2
Ok to proceed? (y) 
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.179.56:8501

  Stopping...
^C
